In [1]:
import wandb

import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist, mnist


(trainIn, trainOut), (testIn, testOut) = fashion_mnist.load_data()

N_train_full = trainOut.shape[0]
N_train = int(0.9*N_train_full)
N_validation = int(0.1 * trainOut.shape[0])
N_test = testOut.shape[0]


idx  = np.random.choice(trainOut.shape[0], N_train_full, replace=False)
idx2 = np.random.choice(testOut.shape[0], N_test, replace=False)

trainInFull = trainIn[idx, :]
trainOutFull = trainOut[idx]

trainIn = trainInFull[:N_train,:]
trainOut = trainOutFull[:N_train]

validIn = trainInFull[N_train:, :]
validOut = trainOutFull[N_train:]    

testIn = testIn[idx2, :]
testOut = testOut[idx2]

In [8]:
sweep_config = {
      "name": "Bayesian Sweep",
      "method": "bayes",
      "metric":{"name": "validation_accuracy","goal": "maximize"},
      "parameters": {"max_epochs": {"values": [5, 10]},"hidden_layers": {"values": [3, 4, 5]},
                    "no_of_hidden_neurons": {"values": [32, 64, 128]},"weight_decay": {"values": [0, 0.0005,0.5]},
                    "learning_rate": {"values": [0.001, 0.0001]},"optimizer": {"values": ["SGD", "MGD", "NAG", "RMSPROP", "ADAM","NADAM"]},
                    "batch_size": {"values": [16, 32, 64]},"initializer": {"values": ["RANDOM", "XAVIER"]},
                    "activation": {"values": [ 'tanh',  'sigmoid', 'ReLU']}}
    }

In [9]:
sweep_id = wandb.sweep(sweep_config,project='fashion-mnist-visualization')


Create sweep with ID: drc6how0
Sweep URL: https://wandb.ai/ma23c011-indian-institute-of-technology-madras/fashion-mnist-visualization/sweeps/drc6how0


In [10]:
from neural_network import NeuralNetwork
def train():    
    config_defaults = dict(
            hidden_layers=4,
            no_of_hidden_neurons=32,
            optimizer="ADAM",
            activation="ReLU",
            learning_rate=1e-4,
            batch_size=16,
            initializer="XAVIER",
            max_epochs= 10,
            weight_decay=0,
            loss="MSE",
        )
        
    wandb.init(config = config_defaults)


    wandb.run.name = "hl_" + str(wandb.config.hidden_layers) + "_hn_" + str(wandb.config.no_of_hidden_neurons) + "_opt_" + wandb.config.optimizer + "_act_" + wandb.config.activation + "_lr_" + str(wandb.config.learning_rate) + "_bs_"+str(wandb.config.batch_size) + "_init_" + wandb.config.initializer + "_ep_"+ str(wandb.config.max_epochs)+ "_l2_" + str(wandb.config.weight_decay) 
    CONFIG = wandb.config
    
    FFNN = NeuralNetwork(
        hidden_layers=CONFIG.hidden_layers,
        no_of_hidden_neurons=CONFIG.no_of_hidden_neurons,
        train_In=trainIn,
        train_out=trainOut,
        N_train = N_train,
        X_val_In = validIn,
        val_out = validOut,
        N_val = N_validation,
        test_In = testIn,
        test_out = testOut,
        N_test = N_test,
        optimizer = CONFIG.optimizer,
        batch_size = CONFIG.batch_size,
        weight_decay = CONFIG.weight_decay,
        learning_rate = CONFIG.learning_rate,
        max_epochs = CONFIG.max_epochs,
        activation = CONFIG.activation,
        initializer = CONFIG.initializer,
        loss = CONFIG.loss
        )


    training_loss, training_accuracy, validation_loss, validation_accuracy, Y_test_pred = FFNN.optimizer(FFNN.max_epochs, FFNN.N_train, FFNN.batch_size, FFNN.learning_rate)
    print(f"Accuracy & Loss on training data is {training_accuracy[FFNN.max_epochs-1]} & {training_loss[FFNN.max_epochs-1]} and Accuracy & Loss on validation data is {validation_accuracy[FFNN.max_epochs-1]} & {validation_loss[FFNN.max_epochs-1]}")
    Y_test_pred = FFNN.predict(FFNN.test_In,FFNN.N_test)
    testing_accuracy = FFNN.accuracy(FFNN.Y_test,Y_test_pred,FFNN.N_test)
    print(f"Accuracy on testing data is {testing_accuracy[0]}")
    return Y_test_pred

 

In [9]:
if __name__ == "__main__":
    train()

Epoch: 0, Loss: 5.010e-02, Training accuracy:0.845, Validation Loss: 4.375e-02, Validation Accuracy: 0.836, Time: 26.634, Learning Rate: 1.000e-04
Epoch: 1, Loss: 3.545e-02, Training accuracy:0.865, Validation Loss: 3.938e-02, Validation Accuracy: 0.855, Time: 42.636, Learning Rate: 1.000e-04
Epoch: 2, Loss: 3.142e-02, Training accuracy:0.884, Validation Loss: 3.502e-02, Validation Accuracy: 0.873, Time: 44.504, Learning Rate: 1.000e-04
Epoch: 3, Loss: 2.912e-02, Training accuracy:0.892, Validation Loss: 3.363e-02, Validation Accuracy: 0.880, Time: 36.046, Learning Rate: 1.000e-04
Epoch: 4, Loss: 2.751e-02, Training accuracy:0.895, Validation Loss: 3.309e-02, Validation Accuracy: 0.883, Time: 34.534, Learning Rate: 1.000e-04
Epoch: 5, Loss: 2.633e-02, Training accuracy:0.900, Validation Loss: 3.253e-02, Validation Accuracy: 0.885, Time: 35.180, Learning Rate: 1.000e-04
Epoch: 6, Loss: 2.533e-02, Training accuracy:0.903, Validation Loss: 3.237e-02, Validation Accuracy: 0.885, Time: 36.1

In [16]:
wandb.logout()

AttributeError: module 'wandb' has no attribute 'logout'

In [11]:
wandb.agent(sweep_id, train, count=1)

wandb: Agent Starting Run: 1gsbkrgw with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 32
wandb: 	hidden_layers: 4
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 5
wandb: 	no_of_hidden_neurons: 128
wandb: 	optimizer: NADAM
wandb: 	weight_decay: 0.5


Epoch: 0, Loss: 3.383e-02, Training accuracy:0.820, Validation Loss: 2.619e-02, Validation Accuracy: 0.825, Time: 87.792, Learning Rate: 1.000e-04
Epoch: 1, Loss: 2.430e-02, Training accuracy:0.843, Validation Loss: 2.336e-02, Validation Accuracy: 0.841, Time: 96.828, Learning Rate: 1.000e-04
Epoch: 2, Loss: 2.224e-02, Training accuracy:0.853, Validation Loss: 2.220e-02, Validation Accuracy: 0.849, Time: 91.168, Learning Rate: 1.000e-04
Epoch: 3, Loss: 2.100e-02, Training accuracy:0.860, Validation Loss: 2.141e-02, Validation Accuracy: 0.855, Time: 85.066, Learning Rate: 1.000e-04
Epoch: 4, Loss: 2.011e-02, Training accuracy:0.864, Validation Loss: 2.099e-02, Validation Accuracy: 0.857, Time: 89.250, Learning Rate: 1.000e-04
Accuracy & Loss on training data is 0.8635925925925926 & 0.02011439985417421 and Accuracy & Loss on validation data is 0.8568333333333333 & 0.020990664330681332
Accuracy on testing data is 0.8479


epoch,▁▃▅▆█
loss,█▃▂▁▁
training_accuracy,▁▅▆▇█
validation_accuracy,▁▅▆██
validation_loss,█▄▃▂▁
epoch,4
loss,0.02011
training_accuracy,0.86359
validation_accuracy,0.85683
validation_loss,0.02099


In [ ]:
wandb.agent(sweep_id, train, count=1)

wandb: Agent Starting Run: jedg30hg with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 16
wandb: 	hidden_layers: 5
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 10
wandb: 	no_of_hidden_neurons: 64
wandb: 	optimizer: ADAM
wandb: 	weight_decay: 0


c:\Users\himan\Documents\GitHub\DA-6401_Assignments\neural_network.py:97: RuntimeWarning: divide by zero encountered in log
  CE = [-Y_true[i] * np.log(Y_pred[i]) for i in range(len(Y_pred))]
c:\Users\himan\Documents\GitHub\DA-6401_Assignments\neural_network.py:97: RuntimeWarning: invalid value encountered in multiply
  CE = [-Y_true[i] * np.log(Y_pred[i]) for i in range(len(Y_pred))]
c:\Users\himan\Documents\GitHub\DA-6401_Assignments\imp_function.py:29: RuntimeWarning: overflow encountered in exp
  return np.exp(Z) / np.sum(np.exp(Z))
c:\Users\himan\Documents\GitHub\DA-6401_Assignments\imp_function.py:29: RuntimeWarning: invalid value encountered in divide
  return np.exp(Z) / np.sum(np.exp(Z))


Epoch: 0, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.099, Time: 32.475, Learning Rate: 1.000e-03
Epoch: 1, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.099, Time: 35.546, Learning Rate: 1.000e-03


In [14]:
wandb.agent(sweep_id, train, count=1)

wandb: Agent Starting Run: 8wiblhyh with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 32
wandb: 	hidden_layers: 4
wandb: 	initializer: RANDOM
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 10
wandb: 	no_of_hidden_neurons: 32
wandb: 	optimizer: NADAM
wandb: 	weight_decay: 0.5


c:\Users\himan\Documents\GitHub\DA-6401_Assignments\imp_function.py:29: RuntimeWarning: overflow encountered in exp
  return np.exp(Z) / np.sum(np.exp(Z))
c:\Users\himan\Documents\GitHub\DA-6401_Assignments\imp_function.py:29: RuntimeWarning: invalid value encountered in divide
  return np.exp(Z) / np.sum(np.exp(Z))
c:\Users\himan\Documents\GitHub\DA-6401_Assignments\neural_network.py:97: RuntimeWarning: divide by zero encountered in log
  CE = [-Y_true[i] * np.log(Y_pred[i]) for i in range(len(Y_pred))]
c:\Users\himan\Documents\GitHub\DA-6401_Assignments\neural_network.py:97: RuntimeWarning: invalid value encountered in multiply
  CE = [-Y_true[i] * np.log(Y_pred[i]) for i in range(len(Y_pred))]


Epoch: 0, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.099, Time: 50.812, Learning Rate: 1.000e-04
Epoch: 1, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.099, Time: 19.792, Learning Rate: 1.000e-04
Epoch: 2, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.099, Time: 25.068, Learning Rate: 1.000e-04
Epoch: 3, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.099, Time: 24.306, Learning Rate: 1.000e-04
Epoch: 4, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.099, Time: 35.522, Learning Rate: 1.000e-04
Epoch: 5, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.099, Time: 23.714, Learning Rate: 1.000e-04
Epoch: 6, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.099, Time: 23.805, Learning Rate: 1.000e-04
Epoch: 7, Loss: nan, Training accuracy:0.100, Validatio

epoch,▁▂▃▃▄▅▆▆▇█
training_accuracy,▁▁▁▁▁▁▁▁▁▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▁
epoch,9
loss,nan
training_accuracy,0.10007
validation_accuracy,0.09933
validation_loss,nan


In [8]:
train()

Epoch: 0, Loss: 6.064e-02, Training accuracy:0.830, Validation Loss: 4.783e-02, Validation Accuracy: 0.828, Time: 86.725, Learning Rate: 1.000e-04
Epoch: 1, Loss: 4.245e-02, Training accuracy:0.856, Validation Loss: 4.179e-02, Validation Accuracy: 0.853, Time: 89.947, Learning Rate: 1.000e-04
Epoch: 2, Loss: 3.863e-02, Training accuracy:0.868, Validation Loss: 3.878e-02, Validation Accuracy: 0.862, Time: 90.070, Learning Rate: 1.000e-04
Epoch: 3, Loss: 3.629e-02, Training accuracy:0.876, Validation Loss: 3.695e-02, Validation Accuracy: 0.869, Time: 88.116, Learning Rate: 1.000e-04
Epoch: 4, Loss: 3.456e-02, Training accuracy:0.880, Validation Loss: 3.580e-02, Validation Accuracy: 0.874, Time: 82.112, Learning Rate: 1.000e-04
Epoch: 5, Loss: 3.320e-02, Training accuracy:0.884, Validation Loss: 3.485e-02, Validation Accuracy: 0.877, Time: 83.057, Learning Rate: 1.000e-04
Epoch: 6, Loss: 3.205e-02, Training accuracy:0.888, Validation Loss: 3.415e-02, Validation Accuracy: 0.878, Time: 89.2

In [5]:
wandb.agent(sweep_id, train, count = 1)

wandb: Agent Starting Run: lqn9ncu8 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	hidden_layers: 3
wandb: 	initializer: RANDOM
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 10
wandb: 	no_of_hidden_neurons: 32
wandb: 	optimizer: MGD
wandb: 	weight_decay: 0
wandb: Currently logged in as: ma23c011 (ma23c011-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch: 0, Loss: 2.455e-01, Training accuracy:0.532, Validation Loss: 1.445e-01, Validation Accuracy: 0.532, Time: 16.549, Learning Rate: 1.000e-03
Epoch: 1, Loss: 1.288e-01, Training accuracy:0.605, Validation Loss: 1.148e-01, Validation Accuracy: 0.607, Time: 17.735, Learning Rate: 1.000e-03
Epoch: 2, Loss: 1.092e-01, Training accuracy:0.634, Validation Loss: 1.033e-01, Validation Accuracy: 0.633, Time: 26.807, Learning Rate: 1.000e-03
Epoch: 3, Loss: 1.004e-01, Training accuracy:0.650, Validation Loss: 9.818e-02, Validation Accuracy: 0.646, Time: 32.240, Learning Rate: 1.000e-03
Epoch: 4, Loss: 9.506e-02, Training accuracy:0.665, Validation Loss: 9.456e-02, Validation Accuracy: 0.663, Time: 28.718, Learning Rate: 1.000e-03
Epoch: 5, Loss: 9.130e-02, Training accuracy:0.675, Validation Loss: 9.139e-02, Validation Accuracy: 0.674, Time: 30.107, Learning Rate: 1.000e-03
Epoch: 6, Loss: 8.834e-02, Training accuracy:0.683, Validation Loss: 8.908e-02, Validation Accuracy: 0.678, Time: 25.4

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▁▁▁▁▁
training_accuracy,▁▄▅▆▆▇▇▇██
validation_accuracy,▁▄▅▆▇▇▇▇██
validation_loss,█▅▃▃▂▂▂▁▁▁
epoch,9
loss,0.08226
training_accuracy,0.70191
validation_accuracy,0.69483
validation_loss,0.08392


In [18]:
wandb.agent(sweep_id, train, count = 1)

wandb: Agent Starting Run: n3ap4l79 with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 64
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 5
wandb: 	num_hidden_neurons: 32
wandb: 	num_layers: 3
wandb: 	optimizer: MGD
wandb: 	weight_decay: 0.5


Epoch: 0, Loss: 1.137e-01, Training accuracy:0.755, Validation Accuracy: 0.748, Time: 21.384, Learning Rate: 1.000e-03
Epoch: 1, Loss: 6.240e-02, Training accuracy:0.804, Validation Accuracy: 0.802, Time: 23.699, Learning Rate: 1.000e-03
Epoch: 2, Loss: 5.411e-02, Training accuracy:0.823, Validation Accuracy: 0.821, Time: 25.473, Learning Rate: 1.000e-03
Epoch: 3, Loss: 5.014e-02, Training accuracy:0.833, Validation Accuracy: 0.830, Time: 28.736, Learning Rate: 1.000e-03
Epoch: 4, Loss: 4.764e-02, Training accuracy:0.839, Validation Accuracy: 0.838, Time: 25.685, Learning Rate: 1.000e-03


epoch,▁▃▅▆█
loss,█▃▂▁▁
training_accuracy,▁▅▇▇█
validation_accuracy,▁▅▇▇█
epoch,4
loss,0.04764
training_accuracy,0.83935
validation_accuracy,0.838


In [19]:
wandb.agent(sweep_id, train, count = 5)

wandb: Agent Starting Run: yke4gnku with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 10
wandb: 	num_hidden_neurons: 64
wandb: 	num_layers: 5
wandb: 	optimizer: NADAM
wandb: 	weight_decay: 0


Epoch: 0, Loss: 8.968e-02, Training accuracy:0.801, Validation Accuracy: 0.801, Time: 26.821, Learning Rate: 1.000e-03
Epoch: 1, Loss: 4.906e-02, Training accuracy:0.850, Validation Accuracy: 0.841, Time: 35.559, Learning Rate: 1.000e-03
Epoch: 2, Loss: 4.226e-02, Training accuracy:0.862, Validation Accuracy: 0.854, Time: 24.274, Learning Rate: 1.000e-03
Epoch: 3, Loss: 3.900e-02, Training accuracy:0.870, Validation Accuracy: 0.862, Time: 29.098, Learning Rate: 1.000e-03
Epoch: 4, Loss: 3.687e-02, Training accuracy:0.877, Validation Accuracy: 0.868, Time: 26.231, Learning Rate: 1.000e-03
Epoch: 5, Loss: 3.533e-02, Training accuracy:0.880, Validation Accuracy: 0.872, Time: 26.075, Learning Rate: 1.000e-03
Epoch: 6, Loss: 3.413e-02, Training accuracy:0.883, Validation Accuracy: 0.875, Time: 24.872, Learning Rate: 1.000e-03
Epoch: 7, Loss: 3.316e-02, Training accuracy:0.886, Validation Accuracy: 0.875, Time: 27.754, Learning Rate: 1.000e-03
Epoch: 8, Loss: 3.235e-02, Training accuracy:0.8

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▁▁▁▁▁
training_accuracy,▁▅▆▆▇▇▇███
validation_accuracy,▁▅▆▇▇█████
epoch,9
loss,0.03165
training_accuracy,0.88996
validation_accuracy,0.87417


wandb: Agent Starting Run: pq3tfnwf with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	initializer: RANDOM
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 10
wandb: 	num_hidden_neurons: 32
wandb: 	num_layers: 4
wandb: 	optimizer: MGD
wandb: 	weight_decay: 0.0005


Epoch: 0, Loss: 2.825e-01, Training accuracy:0.173, Validation Accuracy: 0.188, Time: 37.556, Learning Rate: 1.000e-04
Epoch: 1, Loss: 2.122e-01, Training accuracy:0.260, Validation Accuracy: 0.272, Time: 35.331, Learning Rate: 1.000e-04
Epoch: 2, Loss: 1.956e-01, Training accuracy:0.336, Validation Accuracy: 0.345, Time: 32.921, Learning Rate: 1.000e-04
Epoch: 3, Loss: 1.829e-01, Training accuracy:0.396, Validation Accuracy: 0.408, Time: 30.892, Learning Rate: 1.000e-04
Epoch: 4, Loss: 1.721e-01, Training accuracy:0.441, Validation Accuracy: 0.457, Time: 29.459, Learning Rate: 1.000e-04
Epoch: 5, Loss: 1.629e-01, Training accuracy:0.474, Validation Accuracy: 0.489, Time: 32.921, Learning Rate: 1.000e-04
Epoch: 6, Loss: 1.547e-01, Training accuracy:0.498, Validation Accuracy: 0.512, Time: 30.451, Learning Rate: 1.000e-04
Epoch: 7, Loss: 1.475e-01, Training accuracy:0.516, Validation Accuracy: 0.527, Time: 26.083, Learning Rate: 1.000e-04
Epoch: 8, Loss: 1.411e-01, Training accuracy:0.5

...

Epoch: 0, Loss: 1.360e-01, Training accuracy:0.696, Validation Accuracy: 0.689, Time: 31.402, Learning Rate: 1.000e-04
Epoch: 1, Loss: 8.643e-02, Training accuracy:0.752, Validation Accuracy: 0.746, Time: 31.318, Learning Rate: 1.000e-04
Epoch: 2, Loss: 7.182e-02, Training accuracy:0.783, Validation Accuracy: 0.774, Time: 24.942, Learning Rate: 1.000e-04
Epoch: 3, Loss: 6.354e-02, Training accuracy:0.799, Validation Accuracy: 0.794, Time: 25.857, Learning Rate: 1.000e-04
Epoch: 4, Loss: 5.831e-02, Training accuracy:0.809, Validation Accuracy: 0.804, Time: 22.927, Learning Rate: 1.000e-04


epoch,▁▃▅▆█
loss,█▄▂▁▁
training_accuracy,▁▄▆▇█
validation_accuracy,▁▄▆▇█
epoch,4
loss,0.05831
training_accuracy,0.80941
validation_accuracy,0.804


wandb: Agent Starting Run: w970y894 with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 64
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 5
wandb: 	num_hidden_neurons: 128
wandb: 	num_layers: 5
wandb: 	optimizer: NAG
wandb: 	weight_decay: 0.0005


Epoch: 0, Loss: 2.421e-01, Training accuracy:0.100, Validation Accuracy: 0.101, Time: 27.351, Learning Rate: 1.000e-04
Epoch: 1, Loss: 2.324e-01, Training accuracy:0.100, Validation Accuracy: 0.101, Time: 27.881, Learning Rate: 1.000e-04
Epoch: 2, Loss: 2.308e-01, Training accuracy:0.100, Validation Accuracy: 0.101, Time: 38.233, Learning Rate: 1.000e-04
Epoch: 3, Loss: 2.305e-01, Training accuracy:0.101, Validation Accuracy: 0.102, Time: 24.643, Learning Rate: 1.000e-04
Epoch: 4, Loss: 2.304e-01, Training accuracy:0.109, Validation Accuracy: 0.106, Time: 56.869, Learning Rate: 1.000e-04


epoch,▁▃▅▆█
loss,█▂▁▁▁
training_accuracy,▁▁▁▂█
validation_accuracy,▁▁▁▂█
epoch,4
loss,0.23036
training_accuracy,0.10861
validation_accuracy,0.106


In [24]:
wandb.agent(sweep_id, train, count = 1)

wandb: Agent Starting Run: vuepspuz with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	hidden_layers: 4
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 10
wandb: 	no_of_hidden_neurons: 64
wandb: 	optimizer: NADAM
wandb: 	weight_decay: 0.5


Epoch: 0, Loss: 7.263e-02, Training accuracy:0.834, Validation Accuracy: 0.831, Time: 30.097, Learning Rate: 1.000e-04
Epoch: 1, Loss: 4.531e-02, Training accuracy:0.853, Validation Accuracy: 0.845, Time: 35.721, Learning Rate: 1.000e-04
Epoch: 2, Loss: 4.135e-02, Training accuracy:0.860, Validation Accuracy: 0.854, Time: 40.846, Learning Rate: 1.000e-04
Epoch: 3, Loss: 3.922e-02, Training accuracy:0.866, Validation Accuracy: 0.859, Time: 38.524, Learning Rate: 1.000e-04
Epoch: 4, Loss: 3.778e-02, Training accuracy:0.869, Validation Accuracy: 0.861, Time: 65.374, Learning Rate: 1.000e-04
Epoch: 5, Loss: 3.667e-02, Training accuracy:0.872, Validation Accuracy: 0.863, Time: 49.864, Learning Rate: 1.000e-04
Epoch: 6, Loss: 3.578e-02, Training accuracy:0.875, Validation Accuracy: 0.865, Time: 37.249, Learning Rate: 1.000e-04
Epoch: 7, Loss: 3.501e-02, Training accuracy:0.877, Validation Accuracy: 0.866, Time: 31.135, Learning Rate: 1.000e-04
Epoch: 8, Loss: 3.435e-02, Training accuracy:0.8

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▂▁▁▁▁
training_accuracy,▁▄▅▆▆▇▇▇██
validation_accuracy,▁▃▅▆▆▇▇▇██
epoch,9
loss,0.03374
training_accuracy,0.88096
validation_accuracy,0.86967


Run vuepspuz errored:
Traceback (most recent call last):
  File "c:\Users\himan\.conda\envs\hj\lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
    self._function()
  File "C:\Users\himan\AppData\Local\Temp\ipykernel_25500\1770519332.py", line 45, in train
    training_loss, training_accuracy, validation_loss, validation_accuracy, Y_pred = FFNN.optimizer(FFNN.max_epochs, FFNN.N_train, FFNN.batch_size, FFNN.learning_rate)
ValueError: not enough values to unpack (expected 5, got 4)

wandb: ERROR Run vuepspuz errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "c:\Users\himan\.conda\envs\hj\lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "C:\Users\himan\AppData\Local\Temp\ipykernel_25500\1770519332.py", line 45, in train
wandb: ERROR     training_loss, training_accuracy, validation_loss, validation_accuracy, Y_pred = FFNN.optimizer(FFNN.max_epochs, FFNN.N_train, FFNN.batch_siz

In [6]:
wandb.agent(sweep_id, train, count = 1)

wandb: Agent Starting Run: uh4s27d9 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	hidden_layers: 4
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 10
wandb: 	no_of_hidden_neurons: 128
wandb: 	optimizer: SGD
wandb: 	weight_decay: 0.0005
wandb: Currently logged in as: ma23c011 (ma23c011-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch: 0, Loss: 1.509e-01, Training accuracy:0.691, Validation Loss: 1.116e-01, Validation Accuracy: 0.696, Time: 113.437, Learning Rate: 1.000e-03
Epoch: 1, Loss: 9.757e-02, Training accuracy:0.731, Validation Loss: 8.731e-02, Validation Accuracy: 0.735, Time: 133.348, Learning Rate: 1.000e-03
Epoch: 2, Loss: 8.140e-02, Training accuracy:0.749, Validation Loss: 7.650e-02, Validation Accuracy: 0.751, Time: 193.248, Learning Rate: 1.000e-03
Epoch: 3, Loss: 7.314e-02, Training accuracy:0.761, Validation Loss: 7.018e-02, Validation Accuracy: 0.763, Time: 90.551, Learning Rate: 1.000e-03
Epoch: 4, Loss: 6.798e-02, Training accuracy:0.773, Validation Loss: 6.592e-02, Validation Accuracy: 0.775, Time: 149.488, Learning Rate: 1.000e-03
Epoch: 5, Loss: 6.432e-02, Training accuracy:0.783, Validation Loss: 6.274e-02, Validation Accuracy: 0.782, Time: 85.344, Learning Rate: 1.000e-03
Epoch: 6, Loss: 6.150e-02, Training accuracy:0.792, Validation Loss: 6.025e-02, Validation Accuracy: 0.792, Time: 

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▂▂▂▁▁▁▁
training_accuracy,▁▃▄▅▆▆▇▇██
validation_accuracy,▁▃▄▅▆▆▇▇██
validation_loss,█▅▄▃▂▂▂▁▁▁
epoch,9
loss,0.0557
training_accuracy,0.80961
validation_accuracy,0.81117
validation_loss,0.05492


In [7]:
wandb.agent(sweep_id, train, count = 30)

wandb: Agent Starting Run: y5tlff57 with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 32
wandb: 	hidden_layers: 4
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 5
wandb: 	no_of_hidden_neurons: 32
wandb: 	optimizer: NADAM
wandb: 	weight_decay: 0.0005


Epoch: 0, Loss: 2.244e-01, Training accuracy:0.344, Validation Loss: 2.097e-01, Validation Accuracy: 0.342, Time: 21.071, Learning Rate: 1.000e-04
Epoch: 1, Loss: 1.910e-01, Training accuracy:0.368, Validation Loss: 1.715e-01, Validation Accuracy: 0.368, Time: 25.737, Learning Rate: 1.000e-04
Epoch: 2, Loss: 1.576e-01, Training accuracy:0.448, Validation Loss: 1.454e-01, Validation Accuracy: 0.447, Time: 25.704, Learning Rate: 1.000e-04
Epoch: 3, Loss: 1.378e-01, Training accuracy:0.473, Validation Loss: 1.310e-01, Validation Accuracy: 0.469, Time: 25.477, Learning Rate: 1.000e-04
Epoch: 4, Loss: 1.267e-01, Training accuracy:0.500, Validation Loss: 1.222e-01, Validation Accuracy: 0.489, Time: 25.373, Learning Rate: 1.000e-04


epoch,▁▃▅▆█
loss,█▆▃▂▁
training_accuracy,▁▂▆▇█
validation_accuracy,▁▂▆▇█
validation_loss,█▅▃▂▁
epoch,4
loss,0.1267
training_accuracy,0.50022
validation_accuracy,0.48867
validation_loss,0.12222


wandb: Agent Starting Run: 5yf2mp7o with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 64
wandb: 	hidden_layers: 3
wandb: 	initializer: RANDOM
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 5
wandb: 	no_of_hidden_neurons: 32
wandb: 	optimizer: NADAM
wandb: 	weight_decay: 0.0005


Epoch: 0, Loss: 2.521e-01, Training accuracy:0.445, Validation Loss: 1.761e-01, Validation Accuracy: 0.457, Time: 21.987, Learning Rate: 1.000e-04
Epoch: 1, Loss: 1.568e-01, Training accuracy:0.528, Validation Loss: 1.405e-01, Validation Accuracy: 0.537, Time: 24.206, Learning Rate: 1.000e-04
Epoch: 2, Loss: 1.312e-01, Training accuracy:0.585, Validation Loss: 1.228e-01, Validation Accuracy: 0.590, Time: 23.497, Learning Rate: 1.000e-04
Epoch: 3, Loss: 1.161e-01, Training accuracy:0.629, Validation Loss: 1.108e-01, Validation Accuracy: 0.625, Time: 24.428, Learning Rate: 1.000e-04
Epoch: 4, Loss: 1.056e-01, Training accuracy:0.657, Validation Loss: 1.022e-01, Validation Accuracy: 0.653, Time: 23.818, Learning Rate: 1.000e-04


...

Epoch: 0, Loss: 1.480e-01, Training accuracy:0.658, Validation Loss: 9.040e-02, Validation Accuracy: 0.658, Time: 25.755, Learning Rate: 1.000e-04
Epoch: 1, Loss: 8.018e-02, Training accuracy:0.727, Validation Loss: 7.220e-02, Validation Accuracy: 0.729, Time: 26.810, Learning Rate: 1.000e-04
Epoch: 2, Loss: 6.861e-02, Training accuracy:0.764, Validation Loss: 6.422e-02, Validation Accuracy: 0.764, Time: 27.129, Learning Rate: 1.000e-04
Epoch: 3, Loss: 6.194e-02, Training accuracy:0.785, Validation Loss: 5.907e-02, Validation Accuracy: 0.789, Time: 28.729, Learning Rate: 1.000e-04
Epoch: 4, Loss: 5.752e-02, Training accuracy:0.799, Validation Loss: 5.566e-02, Validation Accuracy: 0.803, Time: 27.302, Learning Rate: 1.000e-04
Epoch: 5, Loss: 5.445e-02, Training accuracy:0.809, Validation Loss: 5.322e-02, Validation Accuracy: 0.815, Time: 27.979, Learning Rate: 1.000e-04
Epoch: 6, Loss: 5.215e-02, Training accuracy:0.818, Validation Loss: 5.136e-02, Validation Accuracy: 0.821, Time: 27.1

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▁▁▁▁▁
training_accuracy,▁▄▅▆▇▇▇███
validation_accuracy,▁▄▅▆▇▇▇███
validation_loss,█▅▄▃▂▂▂▁▁▁
epoch,9
loss,0.04757
training_accuracy,0.83215
validation_accuracy,0.83383
validation_loss,0.04758


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: u486b1lk with config:
wandb: 	activation: tanh
wandb: 	batch_size: 16
wandb: 	hidden_layers: 3
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 10
wandb: 	no_of_hidden_neurons: 128
wandb: 	optimizer: ADAM
wandb: 	weight_decay: 0.5


Epoch: 0, Loss: 4.784e-02, Training accuracy:0.850, Validation Loss: 4.222e-02, Validation Accuracy: 0.845, Time: 71.084, Learning Rate: 1.000e-04
Epoch: 1, Loss: 3.415e-02, Training accuracy:0.880, Validation Loss: 3.670e-02, Validation Accuracy: 0.865, Time: 74.041, Learning Rate: 1.000e-04


: 

In [5]:
wandb.agent(sweep_id, train, count = 30)

wandb: Agent Starting Run: 6sqc7hr5 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	hidden_layers: 4
wandb: 	initializer: RANDOM
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 5
wandb: 	no_of_hidden_neurons: 128
wandb: 	optimizer: RMSPROP
wandb: 	weight_decay: 0.5
wandb: Currently logged in as: ma23c011 (ma23c011-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch: 0, Loss: 1.362e+00, Training accuracy:0.195, Validation Loss: 1.096e+00, Validation Accuracy: 0.191, Time: 160.459, Learning Rate: 1.000e-04
Epoch: 1, Loss: 9.539e-01, Training accuracy:0.288, Validation Loss: 8.411e-01, Validation Accuracy: 0.282, Time: 72.436, Learning Rate: 1.000e-04
Epoch: 2, Loss: 7.474e-01, Training accuracy:0.361, Validation Loss: 6.950e-01, Validation Accuracy: 0.354, Time: 72.131, Learning Rate: 1.000e-04
Epoch: 3, Loss: 6.265e-01, Training accuracy:0.412, Validation Loss: 5.966e-01, Validation Accuracy: 0.397, Time: 70.869, Learning Rate: 1.000e-04
Epoch: 4, Loss: 5.497e-01, Training accuracy:0.448, Validation Loss: 5.281e-01, Validation Accuracy: 0.443, Time: 74.552, Learning Rate: 1.000e-04


epoch,▁▃▅▆█
loss,█▄▃▂▁
training_accuracy,▁▄▆▇█
validation_accuracy,▁▄▆▇█
validation_loss,█▅▃▂▁
epoch,4
loss,0.54971
training_accuracy,0.44813
validation_accuracy,0.443
validation_loss,0.52813


wandb: Agent Starting Run: 22mdm5ee with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 64
wandb: 	hidden_layers: 5
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 10
wandb: 	no_of_hidden_neurons: 32
wandb: 	optimizer: ADAM
wandb: 	weight_decay: 0.5


Epoch: 0, Loss: 6.160e-02, Training accuracy:0.814, Validation Loss: 5.404e-02, Validation Accuracy: 0.807, Time: 23.308, Learning Rate: 1.000e-03
Epoch: 1, Loss: 4.321e-02, Training accuracy:0.844, Validation Loss: 4.549e-02, Validation Accuracy: 0.836, Time: 24.649, Learning Rate: 1.000e-03
Epoch: 2, Loss: 3.860e-02, Training accuracy:0.856, Validation Loss: 4.401e-02, Validation Accuracy: 0.844, Time: 24.214, Learning Rate: 1.000e-03
Epoch: 3, Loss: 3.601e-02, Training accuracy:0.866, Validation Loss: 4.230e-02, Validation Accuracy: 0.855, Time: 24.332, Learning Rate: 1.000e-03
Epoch: 4, Loss: 3.416e-02, Training accuracy:0.875, Validation Loss: 3.928e-02, Validation Accuracy: 0.861, Time: 24.680, Learning Rate: 1.000e-03
Epoch: 5, Loss: 3.273e-02, Training accuracy:0.880, Validation Loss: 3.826e-02, Validation Accuracy: 0.864, Time: 24.458, Learning Rate: 1.000e-03
Epoch: 6, Loss: 3.170e-02, Training accuracy:0.882, Validation Loss: 3.931e-02, Validation Accuracy: 0.862, Time: 24.5

...

Epoch: 0, Loss: 9.438e-02, Training accuracy:0.788, Validation Loss: 6.155e-02, Validation Accuracy: 0.771, Time: 87.436, Learning Rate: 1.000e-04
Epoch: 1, Loss: 5.606e-02, Training accuracy:0.814, Validation Loss: 5.594e-02, Validation Accuracy: 0.796, Time: 90.857, Learning Rate: 1.000e-04
Epoch: 2, Loss: 5.030e-02, Training accuracy:0.825, Validation Loss: 5.337e-02, Validation Accuracy: 0.806, Time: 90.048, Learning Rate: 1.000e-04
Epoch: 3, Loss: 4.728e-02, Training accuracy:0.833, Validation Loss: 5.200e-02, Validation Accuracy: 0.810, Time: 90.927, Learning Rate: 1.000e-04
Epoch: 4, Loss: 4.530e-02, Training accuracy:0.838, Validation Loss: 5.117e-02, Validation Accuracy: 0.815, Time: 95.924, Learning Rate: 1.000e-04
Epoch: 5, Loss: 4.384e-02, Training accuracy:0.843, Validation Loss: 5.048e-02, Validation Accuracy: 0.817, Time: 92.716, Learning Rate: 1.000e-04
Epoch: 6, Loss: 4.268e-02, Training accuracy:0.846, Validation Loss: 4.993e-02, Validation Accuracy: 0.820, Time: 89.0

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▁▁▁▁▁
training_accuracy,▁▄▅▆▆▇▇▇██
validation_accuracy,▁▄▅▆▇▇▇███
validation_loss,█▅▄▃▂▂▂▁▁▁
epoch,9
loss,0.04016
training_accuracy,0.85491
validation_accuracy,0.82583
validation_loss,0.04871


wandb: Agent Starting Run: mjc28enr with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 16
wandb: 	hidden_layers: 3
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 10
wandb: 	no_of_hidden_neurons: 64
wandb: 	optimizer: ADAM
wandb: 	weight_decay: 0


Epoch: 0, Loss: 4.896e-02, Training accuracy:0.858, Validation Loss: 4.188e-02, Validation Accuracy: 0.854, Time: 28.149, Learning Rate: 1.000e-04
Epoch: 1, Loss: 3.495e-02, Training accuracy:0.883, Validation Loss: 3.550e-02, Validation Accuracy: 0.873, Time: 29.433, Learning Rate: 1.000e-04
Epoch: 2, Loss: 3.120e-02, Training accuracy:0.893, Validation Loss: 3.364e-02, Validation Accuracy: 0.877, Time: 29.108, Learning Rate: 1.000e-04
Epoch: 3, Loss: 2.897e-02, Training accuracy:0.898, Validation Loss: 3.271e-02, Validation Accuracy: 0.881, Time: 28.663, Learning Rate: 1.000e-04
Epoch: 4, Loss: 2.748e-02, Training accuracy:0.901, Validation Loss: 3.221e-02, Validation Accuracy: 0.885, Time: 29.054, Learning Rate: 1.000e-04
Epoch: 5, Loss: 2.631e-02, Training accuracy:0.904, Validation Loss: 3.210e-02, Validation Accuracy: 0.885, Time: 28.332, Learning Rate: 1.000e-04
Epoch: 6, Loss: 2.533e-02, Training accuracy:0.906, Validation Loss: 3.220e-02, Validation Accuracy: 0.888, Time: 28.8

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▃▂▂▂▁▁▁
training_accuracy,▁▄▅▆▆▇▇▇██
validation_accuracy,▁▅▆▆▇▇████
validation_loss,█▃▂▁▁▁▁▁▁▁
epoch,9
loss,0.02318
training_accuracy,0.91309
validation_accuracy,0.8885
validation_loss,0.03219


In [6]:
wandb.agent(sweep_id, train, count = 30)

wandb: Agent Starting Run: g1f4h90n with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 16
wandb: 	hidden_layers: 3
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 10
wandb: 	no_of_hidden_neurons: 32
wandb: 	optimizer: NADAM
wandb: 	weight_decay: 0


Epoch: 0, Loss: 5.197e-02, Training accuracy:0.855, Validation Loss: 4.252e-02, Validation Accuracy: 0.847, Time: 20.964, Learning Rate: 1.000e-03
Epoch: 1, Loss: 3.852e-02, Training accuracy:0.871, Validation Loss: 3.797e-02, Validation Accuracy: 0.859, Time: 22.006, Learning Rate: 1.000e-03
Epoch: 2, Loss: 3.483e-02, Training accuracy:0.879, Validation Loss: 3.631e-02, Validation Accuracy: 0.866, Time: 22.032, Learning Rate: 1.000e-03
Epoch: 3, Loss: 3.276e-02, Training accuracy:0.884, Validation Loss: 3.595e-02, Validation Accuracy: 0.864, Time: 22.007, Learning Rate: 1.000e-03
Epoch: 4, Loss: 3.124e-02, Training accuracy:0.888, Validation Loss: 3.569e-02, Validation Accuracy: 0.867, Time: 22.200, Learning Rate: 1.000e-03
Epoch: 5, Loss: 3.004e-02, Training accuracy:0.891, Validation Loss: 3.536e-02, Validation Accuracy: 0.868, Time: 22.384, Learning Rate: 1.000e-03
Epoch: 6, Loss: 2.908e-02, Training accuracy:0.893, Validation Loss: 3.519e-02, Validation Accuracy: 0.871, Time: 23.0

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▃▂▂▂▁▁▁
training_accuracy,▁▄▅▆▆▇▇▇██
validation_accuracy,▁▄▆▆▇▇█▇██
validation_loss,█▄▂▂▁▁▁▁▁▁
epoch,9
loss,0.02689
training_accuracy,0.89844
validation_accuracy,0.87133
validation_loss,0.03568


wandb: Agent Starting Run: dbvjj47x with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	hidden_layers: 4
wandb: 	initializer: RANDOM
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 5
wandb: 	no_of_hidden_neurons: 64
wandb: 	optimizer: NADAM
wandb: 	weight_decay: 0.0005


Epoch: 0, Loss: 7.468e-02, Training accuracy:0.808, Validation Loss: 5.476e-02, Validation Accuracy: 0.803, Time: 29.148, Learning Rate: 1.000e-03
Epoch: 1, Loss: 5.001e-02, Training accuracy:0.832, Validation Loss: 5.011e-02, Validation Accuracy: 0.818, Time: 29.858, Learning Rate: 1.000e-03
Epoch: 2, Loss: 4.459e-02, Training accuracy:0.846, Validation Loss: 4.708e-02, Validation Accuracy: 0.830, Time: 29.908, Learning Rate: 1.000e-03
Epoch: 3, Loss: 4.147e-02, Training accuracy:0.853, Validation Loss: 4.533e-02, Validation Accuracy: 0.837, Time: 30.122, Learning Rate: 1.000e-03
Epoch: 4, Loss: 3.931e-02, Training accuracy:0.860, Validation Loss: 4.429e-02, Validation Accuracy: 0.842, Time: 29.393, Learning Rate: 1.000e-03


...

Epoch: 0, Loss: 1.681e-01, Training accuracy:0.652, Validation Loss: 9.933e-02, Validation Accuracy: 0.647, Time: 29.497, Learning Rate: 1.000e-04
Epoch: 1, Loss: 8.742e-02, Training accuracy:0.711, Validation Loss: 8.076e-02, Validation Accuracy: 0.704, Time: 29.746, Learning Rate: 1.000e-04
Epoch: 2, Loss: 7.599e-02, Training accuracy:0.738, Validation Loss: 7.365e-02, Validation Accuracy: 0.732, Time: 30.482, Learning Rate: 1.000e-04
Epoch: 3, Loss: 7.018e-02, Training accuracy:0.755, Validation Loss: 6.927e-02, Validation Accuracy: 0.745, Time: 31.388, Learning Rate: 1.000e-04
Epoch: 4, Loss: 6.628e-02, Training accuracy:0.767, Validation Loss: 6.621e-02, Validation Accuracy: 0.758, Time: 29.805, Learning Rate: 1.000e-04
Epoch: 5, Loss: 6.342e-02, Training accuracy:0.775, Validation Loss: 6.395e-02, Validation Accuracy: 0.764, Time: 29.629, Learning Rate: 1.000e-04
Epoch: 6, Loss: 6.123e-02, Training accuracy:0.782, Validation Loss: 6.222e-02, Validation Accuracy: 0.772, Time: 29.4

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▁▁▁▁▁
training_accuracy,▁▄▅▆▇▇▇███
validation_accuracy,▁▄▅▆▇▇▇███
validation_loss,█▅▄▃▂▂▂▁▁▁
epoch,9
loss,0.05673
training_accuracy,0.79639
validation_accuracy,0.786
validation_loss,0.05863


wandb: Agent Starting Run: xrd7ba9c with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 32
wandb: 	hidden_layers: 4
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 10
wandb: 	no_of_hidden_neurons: 64
wandb: 	optimizer: ADAM
wandb: 	weight_decay: 0.0005


Epoch: 0, Loss: 5.144e-02, Training accuracy:0.856, Validation Loss: 4.279e-02, Validation Accuracy: 0.846, Time: 30.252, Learning Rate: 1.000e-04
Epoch: 1, Loss: 3.616e-02, Training accuracy:0.873, Validation Loss: 3.753e-02, Validation Accuracy: 0.861, Time: 32.146, Learning Rate: 1.000e-04
Epoch: 2, Loss: 3.230e-02, Training accuracy:0.887, Validation Loss: 3.467e-02, Validation Accuracy: 0.870, Time: 32.192, Learning Rate: 1.000e-04
Epoch: 3, Loss: 3.006e-02, Training accuracy:0.894, Validation Loss: 3.374e-02, Validation Accuracy: 0.873, Time: 32.174, Learning Rate: 1.000e-04
Epoch: 4, Loss: 2.850e-02, Training accuracy:0.897, Validation Loss: 3.329e-02, Validation Accuracy: 0.877, Time: 31.848, Learning Rate: 1.000e-04
Epoch: 5, Loss: 2.733e-02, Training accuracy:0.902, Validation Loss: 3.272e-02, Validation Accuracy: 0.881, Time: 32.542, Learning Rate: 1.000e-04
Epoch: 6, Loss: 2.634e-02, Training accuracy:0.904, Validation Loss: 3.259e-02, Validation Accuracy: 0.881, Time: 32.4

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▃▂▂▂▁▁▁
training_accuracy,▁▃▅▆▆▇▇███
validation_accuracy,▁▄▅▆▇▇▇███
validation_loss,█▅▃▂▂▁▁▁▁▁
epoch,9
loss,0.02422
training_accuracy,0.91148
validation_accuracy,0.88433
validation_loss,0.03229


In [7]:
wandb.agent(sweep_id, train, count = 30)

wandb: Agent Starting Run: ljd9wjuo with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	hidden_layers: 3
wandb: 	initializer: RANDOM
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 10
wandb: 	no_of_hidden_neurons: 128
wandb: 	optimizer: MGD
wandb: 	weight_decay: 0.0005


Epoch: 0, Loss: 1.323e-01, Training accuracy:0.689, Validation Loss: 9.184e-02, Validation Accuracy: 0.684, Time: 63.918, Learning Rate: 1.000e-03
Epoch: 1, Loss: 8.236e-02, Training accuracy:0.732, Validation Loss: 7.936e-02, Validation Accuracy: 0.723, Time: 61.818, Learning Rate: 1.000e-03
Epoch: 2, Loss: 7.328e-02, Training accuracy:0.753, Validation Loss: 7.356e-02, Validation Accuracy: 0.741, Time: 63.950, Learning Rate: 1.000e-03
Epoch: 3, Loss: 6.821e-02, Training accuracy:0.767, Validation Loss: 6.983e-02, Validation Accuracy: 0.751, Time: 62.554, Learning Rate: 1.000e-03
Epoch: 4, Loss: 6.475e-02, Training accuracy:0.777, Validation Loss: 6.716e-02, Validation Accuracy: 0.760, Time: 63.460, Learning Rate: 1.000e-03
Epoch: 5, Loss: 6.215e-02, Training accuracy:0.785, Validation Loss: 6.509e-02, Validation Accuracy: 0.767, Time: 62.223, Learning Rate: 1.000e-03
Epoch: 6, Loss: 6.007e-02, Training accuracy:0.791, Validation Loss: 6.340e-02, Validation Accuracy: 0.774, Time: 62.6

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▃▂▂▂▁▁▁▁
training_accuracy,▁▄▅▆▆▇▇▇██
validation_accuracy,▁▄▅▆▆▇▇▇██
validation_loss,█▅▄▃▃▂▂▁▁▁
epoch,9
loss,0.05562
training_accuracy,0.80435
validation_accuracy,0.78417
validation_loss,0.05975


wandb: Agent Starting Run: vll3ejts with config:
wandb: 	activation: tanh
wandb: 	batch_size: 16
wandb: 	hidden_layers: 3
wandb: 	initializer: RANDOM
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 5
wandb: 	no_of_hidden_neurons: 64
wandb: 	optimizer: NAG
wandb: 	weight_decay: 0


Epoch: 0, Loss: 2.305e-01, Training accuracy:0.618, Validation Loss: 1.293e-01, Validation Accuracy: 0.615, Time: 29.783, Learning Rate: 1.000e-03
Epoch: 1, Loss: 1.116e-01, Training accuracy:0.672, Validation Loss: 9.922e-02, Validation Accuracy: 0.662, Time: 31.374, Learning Rate: 1.000e-03
Epoch: 2, Loss: 9.182e-02, Training accuracy:0.704, Validation Loss: 8.920e-02, Validation Accuracy: 0.696, Time: 31.311, Learning Rate: 1.000e-03
Epoch: 3, Loss: 8.353e-02, Training accuracy:0.714, Validation Loss: 8.438e-02, Validation Accuracy: 0.710, Time: 31.474, Learning Rate: 1.000e-03
Epoch: 4, Loss: 7.806e-02, Training accuracy:0.730, Validation Loss: 8.054e-02, Validation Accuracy: 0.725, Time: 31.098, Learning Rate: 1.000e-03


...

epoch,▁▃▅▆█
loss,█▄▃▂▁
training_accuracy,▁▁▁▁▁
validation_accuracy,▁▁▁▁▁
validation_loss,█▇▅▃▁
epoch,4
loss,0.22999
training_accuracy,0.10078
validation_accuracy,0.093
validation_loss,0.23003


wandb: Agent Starting Run: 9zgqgbzi with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 32
wandb: 	hidden_layers: 4
wandb: 	initializer: RANDOM
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 5
wandb: 	no_of_hidden_neurons: 128
wandb: 	optimizer: SGD
wandb: 	weight_decay: 0.5


c:\Users\himan\Documents\GitHub\DA-6401_Assignments\imp_function.py:29: RuntimeWarning: overflow encountered in exp
  return np.exp(Z) / np.sum(np.exp(Z))
c:\Users\himan\Documents\GitHub\DA-6401_Assignments\imp_function.py:29: RuntimeWarning: invalid value encountered in divide
  return np.exp(Z) / np.sum(np.exp(Z))
c:\Users\himan\Documents\GitHub\DA-6401_Assignments\neural_network.py:97: RuntimeWarning: divide by zero encountered in log
  CE = [-Y_true[i] * np.log(Y_pred[i]) for i in range(len(Y_pred))]
c:\Users\himan\Documents\GitHub\DA-6401_Assignments\neural_network.py:97: RuntimeWarning: invalid value encountered in multiply
  CE = [-Y_true[i] * np.log(Y_pred[i]) for i in range(len(Y_pred))]


Epoch: 0, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 74.020, Learning Rate: 1.000e-04
Epoch: 1, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 74.012, Learning Rate: 1.000e-04
Epoch: 2, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 74.266, Learning Rate: 1.000e-04
Epoch: 3, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 75.173, Learning Rate: 1.000e-04
Epoch: 4, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 74.149, Learning Rate: 1.000e-04


epoch,▁▃▅▆█
training_accuracy,▁▁▁▁▁
validation_accuracy,▁▁▁▁▁
epoch,4
loss,nan
training_accuracy,0.09994
validation_accuracy,0.1005
validation_loss,nan


In [8]:
wandb.agent(sweep_id, train, count = 30)

wandb: Agent Starting Run: 0pfe8im6 with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 16
wandb: 	hidden_layers: 4
wandb: 	initializer: RANDOM
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 10
wandb: 	no_of_hidden_neurons: 32
wandb: 	optimizer: ADAM
wandb: 	weight_decay: 0.0005


c:\Users\himan\Documents\GitHub\DA-6401_Assignments\neural_network.py:97: RuntimeWarning: divide by zero encountered in log
  CE = [-Y_true[i] * np.log(Y_pred[i]) for i in range(len(Y_pred))]
c:\Users\himan\Documents\GitHub\DA-6401_Assignments\neural_network.py:97: RuntimeWarning: invalid value encountered in multiply
  CE = [-Y_true[i] * np.log(Y_pred[i]) for i in range(len(Y_pred))]
c:\Users\himan\Documents\GitHub\DA-6401_Assignments\imp_function.py:29: RuntimeWarning: overflow encountered in exp
  return np.exp(Z) / np.sum(np.exp(Z))
c:\Users\himan\Documents\GitHub\DA-6401_Assignments\imp_function.py:29: RuntimeWarning: invalid value encountered in divide
  return np.exp(Z) / np.sum(np.exp(Z))


Epoch: 0, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 23.271, Learning Rate: 1.000e-03
Epoch: 1, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 23.806, Learning Rate: 1.000e-03
Epoch: 2, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 23.929, Learning Rate: 1.000e-03
Epoch: 3, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 24.003, Learning Rate: 1.000e-03
Epoch: 4, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 23.918, Learning Rate: 1.000e-03
Epoch: 5, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 23.909, Learning Rate: 1.000e-03
Epoch: 6, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 23.551, Learning Rate: 1.000e-03
Epoch: 7, Loss: nan, Training accuracy:0.100, Validatio

epoch,▁▂▃▃▄▅▆▆▇█
training_accuracy,▁▁▁▁▁▁▁▁▁▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▁
epoch,9
loss,nan
training_accuracy,0.09994
validation_accuracy,0.1005
validation_loss,nan


wandb: Agent Starting Run: ajeslya1 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	hidden_layers: 3
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 10
wandb: 	no_of_hidden_neurons: 32
wandb: 	optimizer: ADAM
wandb: 	weight_decay: 0.5


...

Epoch: 0, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 225.159, Learning Rate: 1.000e-04
Epoch: 1, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 222.592, Learning Rate: 1.000e-04
Epoch: 2, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 222.116, Learning Rate: 1.000e-04
Epoch: 3, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 224.102, Learning Rate: 1.000e-04
Epoch: 4, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 225.101, Learning Rate: 1.000e-04
Epoch: 5, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 223.952, Learning Rate: 1.000e-04
Epoch: 6, Loss: nan, Training accuracy:0.100, Validation Loss: nan, Validation Accuracy: 0.101, Time: 224.357, Learning Rate: 1.000e-04
Epoch: 7, Loss: nan, Training accuracy:0.100, Va

epoch,▁▂▃▃▄▅▆▆▇█
training_accuracy,▁▁▁▁▁▁▁▁▁▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▁
epoch,9
loss,nan
training_accuracy,0.09994
validation_accuracy,0.1005
validation_loss,nan


wandb: Agent Starting Run: f6c9koyz with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 64
wandb: 	hidden_layers: 3
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 5
wandb: 	no_of_hidden_neurons: 128
wandb: 	optimizer: ADAM
wandb: 	weight_decay: 0.0005


Epoch: 0, Loss: 7.074e-02, Training accuracy:0.837, Validation Loss: 4.731e-02, Validation Accuracy: 0.828, Time: 193.719, Learning Rate: 1.000e-04
Epoch: 1, Loss: 4.176e-02, Training accuracy:0.856, Validation Loss: 4.288e-02, Validation Accuracy: 0.848, Time: 190.405, Learning Rate: 1.000e-04
Epoch: 2, Loss: 3.818e-02, Training accuracy:0.865, Validation Loss: 4.045e-02, Validation Accuracy: 0.855, Time: 191.349, Learning Rate: 1.000e-04
Epoch: 3, Loss: 3.622e-02, Training accuracy:0.870, Validation Loss: 3.884e-02, Validation Accuracy: 0.862, Time: 191.325, Learning Rate: 1.000e-04
Epoch: 4, Loss: 3.489e-02, Training accuracy:0.875, Validation Loss: 3.784e-02, Validation Accuracy: 0.864, Time: 191.002, Learning Rate: 1.000e-04


epoch,▁▃▅▆█
loss,█▂▂▁▁
training_accuracy,▁▅▆▇█
validation_accuracy,▁▅▆██
validation_loss,█▅▃▂▁
epoch,4
loss,0.03489
training_accuracy,0.87487
validation_accuracy,0.864
validation_loss,0.03784


In [ ]:
wandb.agent(sweep_id, train, count = 30)

In [1]:
from keras.datasets import mnist
import wandb
import numpy as np


(trainIn, trainOut), (testIn, testOut) = mnist.load_data()

N_train_full = trainOut.shape[0]
N_train = int(0.9*N_train_full)
N_validation = int(0.1 * trainOut.shape[0])
N_test = testOut.shape[0]


idx  = np.random.choice(trainOut.shape[0], N_train_full, replace=False)
idx2 = np.random.choice(testOut.shape[0], N_test, replace=False)

trainInFull = trainIn[idx, :]
trainOutFull = trainOut[idx]

trainIn = trainInFull[:N_train,:]
trainOut = trainOutFull[:N_train]

validIn = trainInFull[N_train:, :]
validOut = trainOutFull[N_train:]    

testIn = testIn[idx2, :]
testOut = testOut[idx2]

In [9]:
sweep_config = {
  "name": "Bayesian Sweep",
  "method": "bayes",
  "metric":{
  "name": "validationaccuracy",
  "goal": "maximize"
  },
  "parameters": {
        "max_epochs": {"values": [2, 5, 10]},

        "hidden_layers": {"values": [3, 4, 5]},
        
        "no_of_hidden_neurons": {"values": [32, 64, 128]},
        
        "weight_decay": {"values": [0, 0.0005,0.5]},
        
        "learning_rate": {"values": [0.001, 0.0001]},
        
        "optimizer": {"values": ["SGD"]},
                    
        "batch_size": {"values": [16, 32, 64]},

        "initializer": {"values": ["RANDOM", "XAVIER"]},
        
        "activation": {"values": [ 'tanh',  'sigmoid', 'ReLU']}
        }
    }

sweep_id = wandb.sweep(sweep_config,project='mnist')

Create sweep with ID: gu9uwgrg
Sweep URL: https://wandb.ai/ma23c011-indian-institute-of-technology-madras/mnist/sweeps/gu9uwgrg


In [10]:
from neural_network import NeuralNetwork
def train():    
    config_defaults = dict(
            max_epochs= 5,
            hidden_layers=3,
            no_of_hidden_neurons=64,
            weight_decay=0,
            learning_rate=1e-4,
            optimizer="SGD",
            batch_size=32,
            activation="ReLU",
            initializer="XAVIER",
            loss="CROSS",
        )
        
    wandb.init(config = config_defaults, project='mnist')


    wandb.run.name = "hl_" + str(wandb.config.hidden_layers) + "_hn_" + str(wandb.config.no_of_hidden_neurons) + "_opt_" + wandb.config.optimizer + "_act_" + wandb.config.activation + "_lr_" + str(wandb.config.learning_rate) + "_bs_"+str(wandb.config.batch_size) + "_init_" + wandb.config.initializer + "_ep_"+ str(wandb.config.max_epochs)+ "_l2_" + str(wandb.config.weight_decay) 
    CONFIG = wandb.config
    
    FFNN = NeuralNetwork(
        hidden_layers=CONFIG.hidden_layers,
        no_of_hidden_neurons=CONFIG.no_of_hidden_neurons,
        train_In=trainIn,
        train_out=trainOut,
        N_train = N_train,
        X_val_In = validIn,
        val_out = validOut,
        N_val = N_validation,
        test_In = testIn,
        test_out = testOut,
        N_test = N_test,
        optimizer = CONFIG.optimizer,
        batch_size = CONFIG.batch_size,
        weight_decay = CONFIG.weight_decay,
        learning_rate = CONFIG.learning_rate,
        max_epochs = CONFIG.max_epochs,
        activation = CONFIG.activation,
        initializer = CONFIG.initializer,
        loss = CONFIG.loss
        )


    training_loss, training_accuracy, validation_loss, validation_accuracy, Y_pred = FFNN.optimizer(FFNN.max_epochs, FFNN.N_train, FFNN.batch_size, FFNN.learning_rate)
 

In [5]:
wandb.agent(sweep_id, train, count = 1)

wandb: Agent Starting Run: custfuek with config:
wandb: 	activation: tanh
wandb: 	batch_size: 16
wandb: 	hidden_layers: 4
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 5
wandb: 	no_of_hidden_neurons: 32
wandb: 	optimizer: NADAM
wandb: 	weight_decay: 0
wandb: Currently logged in as: ma23c011 (ma23c011-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch: 0, Loss: 2.958e-02, Training accuracy:0.946, Validation Loss: 2.066e-02, Validation Accuracy: 0.937, Time: 17.751, Learning Rate: 1.000e-03
Epoch: 1, Loss: 1.491e-02, Training accuracy:0.960, Validation Loss: 1.655e-02, Validation Accuracy: 0.951, Time: 22.211, Learning Rate: 1.000e-03
Epoch: 2, Loss: 1.158e-02, Training accuracy:0.967, Validation Loss: 1.486e-02, Validation Accuracy: 0.956, Time: 21.497, Learning Rate: 1.000e-03
Epoch: 3, Loss: 9.689e-03, Training accuracy:0.972, Validation Loss: 1.412e-02, Validation Accuracy: 0.958, Time: 21.835, Learning Rate: 1.000e-03
Epoch: 4, Loss: 8.358e-03, Training accuracy:0.975, Validation Loss: 1.382e-02, Validation Accuracy: 0.959, Time: 21.462, Learning Rate: 1.000e-03


epoch,▁▃▅▆█
loss,█▃▂▁▁
training_accuracy,▁▄▆▇█
validation_accuracy,▁▅▇██
validation_loss,█▄▂▁▁
epoch,4
loss,0.00836
training_accuracy,0.9745
validation_accuracy,0.95867
validation_loss,0.01382


In [6]:
wandb.agent(sweep_id, train, count = 5)

wandb: Agent Starting Run: ujkai6cf with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	hidden_layers: 4
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 5
wandb: 	no_of_hidden_neurons: 64
wandb: 	optimizer: RMSPROP
wandb: 	weight_decay: 0.0005


Epoch: 0, Loss: 2.023e-01, Training accuracy:0.582, Validation Loss: 1.439e-01, Validation Accuracy: 0.569, Time: 72.867, Learning Rate: 1.000e-04
Epoch: 1, Loss: 1.049e-01, Training accuracy:0.769, Validation Loss: 8.235e-02, Validation Accuracy: 0.763, Time: 74.627, Learning Rate: 1.000e-04
Epoch: 2, Loss: 6.904e-02, Training accuracy:0.846, Validation Loss: 5.983e-02, Validation Accuracy: 0.841, Time: 75.713, Learning Rate: 1.000e-04
Epoch: 3, Loss: 5.226e-02, Training accuracy:0.879, Validation Loss: 4.727e-02, Validation Accuracy: 0.874, Time: 77.796, Learning Rate: 1.000e-04
Epoch: 4, Loss: 4.240e-02, Training accuracy:0.897, Validation Loss: 3.993e-02, Validation Accuracy: 0.890, Time: 72.966, Learning Rate: 1.000e-04


epoch,▁▃▅▆█
loss,█▄▂▁▁
training_accuracy,▁▅▇██
validation_accuracy,▁▅▇██
validation_loss,█▄▂▁▁
epoch,4
loss,0.0424
training_accuracy,0.89717
validation_accuracy,0.88983
validation_loss,0.03993


wandb: Agent Starting Run: em6o1n3y with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	hidden_layers: 3
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 5
wandb: 	no_of_hidden_neurons: 128
wandb: 	optimizer: RMSPROP
wandb: 	weight_decay: 0.0005


Epoch: 0, Loss: 2.622e-02, Training accuracy:0.955, Validation Loss: 1.691e-02, Validation Accuracy: 0.946, Time: 134.463, Learning Rate: 1.000e-03
Epoch: 1, Loss: 1.239e-02, Training accuracy:0.975, Validation Loss: 1.135e-02, Validation Accuracy: 0.966, Time: 119.159, Learning Rate: 1.000e-03
Epoch: 2, Loss: 8.255e-03, Training accuracy:0.981, Validation Loss: 1.040e-02, Validation Accuracy: 0.969, Time: 117.763, Learning Rate: 1.000e-03
Epoch: 3, Loss: 5.898e-03, Training accuracy:0.985, Validation Loss: 1.013e-02, Validation Accuracy: 0.972, Time: 119.658, Learning Rate: 1.000e-03
Epoch: 4, Loss: 4.333e-03, Training accuracy:0.987, Validation Loss: 9.983e-03, Validation Accuracy: 0.972, Time: 119.677, Learning Rate: 1.000e-03


...

Epoch: 0, Loss: 2.265e-02, Training accuracy:0.966, Validation Loss: 1.322e-02, Validation Accuracy: 0.960, Time: 167.045, Learning Rate: 1.000e-04
Epoch: 1, Loss: 8.734e-03, Training accuracy:0.983, Validation Loss: 8.736e-03, Validation Accuracy: 0.975, Time: 166.984, Learning Rate: 1.000e-04
Epoch: 2, Loss: 5.202e-03, Training accuracy:0.987, Validation Loss: 8.742e-03, Validation Accuracy: 0.977, Time: 169.027, Learning Rate: 1.000e-04
Epoch: 3, Loss: 3.361e-03, Training accuracy:0.992, Validation Loss: 8.984e-03, Validation Accuracy: 0.978, Time: 165.295, Learning Rate: 1.000e-04
Epoch: 4, Loss: 2.291e-03, Training accuracy:0.994, Validation Loss: 9.287e-03, Validation Accuracy: 0.979, Time: 171.760, Learning Rate: 1.000e-04


epoch,▁▃▅▆█
loss,█▃▂▁▁
training_accuracy,▁▅▆▇█
validation_accuracy,▁▆▇▇█
validation_loss,█▁▁▁▂
epoch,4
loss,0.00229
training_accuracy,0.99406
validation_accuracy,0.97933
validation_loss,0.00929


wandb: Agent Starting Run: 7csuro55 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	hidden_layers: 4
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 5
wandb: 	no_of_hidden_neurons: 128
wandb: 	optimizer: RMSPROP
wandb: 	weight_decay: 0.0005


Epoch: 0, Loss: 2.665e-02, Training accuracy:0.956, Validation Loss: 1.620e-02, Validation Accuracy: 0.948, Time: 138.270, Learning Rate: 1.000e-03
Epoch: 1, Loss: 1.312e-02, Training accuracy:0.969, Validation Loss: 1.316e-02, Validation Accuracy: 0.960, Time: 140.956, Learning Rate: 1.000e-03
Epoch: 2, Loss: 9.129e-03, Training accuracy:0.978, Validation Loss: 1.117e-02, Validation Accuracy: 0.967, Time: 113.304, Learning Rate: 1.000e-03
Epoch: 3, Loss: 6.712e-03, Training accuracy:0.981, Validation Loss: 1.093e-02, Validation Accuracy: 0.968, Time: 148.619, Learning Rate: 1.000e-03
Epoch: 4, Loss: 5.042e-03, Training accuracy:0.982, Validation Loss: 1.315e-02, Validation Accuracy: 0.963, Time: 135.296, Learning Rate: 1.000e-03


epoch,▁▃▅▆█
loss,█▄▂▂▁
training_accuracy,▁▄▇██
validation_accuracy,▁▅██▆
validation_loss,█▄▁▁▄
epoch,4
loss,0.00504
training_accuracy,0.98239
validation_accuracy,0.96317
validation_loss,0.01315


In [12]:
wandb.agent(sweep_id, train, count = 1)

wandb: Agent Starting Run: gg7j2x4r with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	hidden_layers: 4
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 5
wandb: 	no_of_hidden_neurons: 64
wandb: 	optimizer: NADAM
wandb: 	weight_decay: 0.5


Epoch: 0, Loss: 2.705e-02, Training accuracy:0.952, Validation Loss: 1.800e-02, Validation Accuracy: 0.946, Time: 26.434, Learning Rate: 1.000e-03
Epoch: 1, Loss: 1.259e-02, Training accuracy:0.971, Validation Loss: 1.263e-02, Validation Accuracy: 0.963, Time: 30.566, Learning Rate: 1.000e-03
Epoch: 2, Loss: 8.831e-03, Training accuracy:0.979, Validation Loss: 1.096e-02, Validation Accuracy: 0.967, Time: 29.595, Learning Rate: 1.000e-03
Epoch: 3, Loss: 6.689e-03, Training accuracy:0.984, Validation Loss: 1.017e-02, Validation Accuracy: 0.970, Time: 31.007, Learning Rate: 1.000e-03
Epoch: 4, Loss: 5.279e-03, Training accuracy:0.986, Validation Loss: 1.003e-02, Validation Accuracy: 0.970, Time: 30.750, Learning Rate: 1.000e-03


epoch,▁▃▅▆█
loss,█▃▂▁▁
training_accuracy,▁▅▇▇█
validation_accuracy,▁▆▇██
validation_loss,█▃▂▁▁
epoch,4
loss,0.00528
training_accuracy,0.98646
validation_accuracy,0.96967
validation_loss,0.01003


In [7]:
wandb.agent(sweep_id, train, count = 1)

wandb: Agent Starting Run: ww0eahgy with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 64
wandb: 	hidden_layers: 5
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 10
wandb: 	no_of_hidden_neurons: 64
wandb: 	optimizer: NADAM
wandb: 	weight_decay: 0.5
wandb: Currently logged in as: ma23c011 (ma23c011-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch: 0, Loss: 8.486e-02, Training accuracy:0.894, Validation Loss: 3.857e-02, Validation Accuracy: 0.893, Time: 28.946, Learning Rate: 1.000e-04
Epoch: 1, Loss: 3.345e-02, Training accuracy:0.915, Validation Loss: 3.005e-02, Validation Accuracy: 0.910, Time: 35.704, Learning Rate: 1.000e-04
Epoch: 2, Loss: 2.773e-02, Training accuracy:0.926, Validation Loss: 2.611e-02, Validation Accuracy: 0.922, Time: 38.171, Learning Rate: 1.000e-04
Epoch: 3, Loss: 2.451e-02, Training accuracy:0.933, Validation Loss: 2.357e-02, Validation Accuracy: 0.930, Time: 41.890, Learning Rate: 1.000e-04
Epoch: 4, Loss: 2.222e-02, Training accuracy:0.939, Validation Loss: 2.172e-02, Validation Accuracy: 0.936, Time: 39.188, Learning Rate: 1.000e-04
Epoch: 5, Loss: 2.044e-02, Training accuracy:0.944, Validation Loss: 2.024e-02, Validation Accuracy: 0.939, Time: 43.328, Learning Rate: 1.000e-04
Epoch: 6, Loss: 1.896e-02, Training accuracy:0.948, Validation Loss: 1.899e-02, Validation Accuracy: 0.943, Time: 36.3

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▁▁▁▁▁
training_accuracy,▁▃▅▅▆▇▇▇██
validation_accuracy,▁▃▄▅▆▇▇███
validation_loss,█▅▄▃▃▂▂▂▁▁
epoch,9
loss,0.01577
training_accuracy,0.95628
validation_accuracy,0.95183
validation_loss,0.01635


In [11]:
train()

Epoch: 0, Loss: 2.638e-02, Training accuracy:0.958, Validation Loss: 1.612e-02, Validation Accuracy: 0.956, Time: 28.653, Learning Rate: 1.000e-04
Epoch: 1, Loss: 1.137e-02, Training accuracy:0.975, Validation Loss: 1.147e-02, Validation Accuracy: 0.969, Time: 29.942, Learning Rate: 1.000e-04
Epoch: 2, Loss: 8.207e-03, Training accuracy:0.981, Validation Loss: 1.030e-02, Validation Accuracy: 0.971, Time: 31.138, Learning Rate: 1.000e-04
Epoch: 3, Loss: 6.506e-03, Training accuracy:0.984, Validation Loss: 9.712e-03, Validation Accuracy: 0.974, Time: 30.704, Learning Rate: 1.000e-04
Epoch: 4, Loss: 5.374e-03, Training accuracy:0.986, Validation Loss: 9.402e-03, Validation Accuracy: 0.974, Time: 31.993, Learning Rate: 1.000e-04
